# Creating functions
- read in base9 dataset
    - create data table of features 
    - creates random forest ml model (creating or using existing model?)
        - still need to remove the certain features (16th, 84th, mean, median, median, binary)
        - ADD 84-med(upp) & med-16 (low)
        - need to export to use later

- Add to 1st
    - read in model
    - read in NEW dataset 
    - apply model to NEW dataset

In [1]:
from create_model import *
#importing functions from .py file

In [2]:
directory = "/Users/justycewatson/Desktop/CIERA_files/NGC2682/jw_output"
#computer directory the files will come from

model_cluster_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
model_cluster_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.219438,1.058627e-263,9824.639407
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.166996,1.506331e-148,10221.965539
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.148112,1.155575e-119,9973.991074
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.140972,1.579208e-107,9526.342453
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.130705,3.307284e-90,9843.150528
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.168000,1.401356e-151,9922.136969
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.157878,8.128708e-134,9711.830805
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.147469,7.906632e-115,10043.330216
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.120905,7.605685e-78,9747.557747


# Creating ML model

In [3]:
##reading in dataframe containing known sampling quality to train model on

df1 = pd.read_csv('/Users/justycewatson/Desktop/CIERA_files/NGC2682/CSV_Files/NGC2682_Age_Stats.csv',sep=',')
sampling_df = df1[df1['Single Sampling'].isna() == False]

- change ngc2682 to broad name (model naming, replacing for source id)

- repolace xtest with 6819 np array
- no y_test
- xcolumns is 6819array columns

- another cell making plot comparing with elizabeths labels like the histogram/bar chart

- function to prepare a df for makepreds, 

In [41]:
X.columns

Index(['Width', 'Upper_bound', 'Lower_bound', 'Stdev', 'SnR', 'Dip_p',
       'Dip_value', 'KS_value', 'KS_p', 'ESS'],
      dtype='object')

In [4]:
X, y, clf, X_test, X_train, y_test = create_model(model_cluster_statistic, sampling_df)
y_pred = make_preds(X_test, clf, y_test = y_test)

Accuracy: 0.9672897196261683
              precision    recall  f1-score   support

         Bad       0.98      0.98      0.98       359
        Good       0.89      0.91      0.90        69

    accuracy                           0.97       428
   macro avg       0.94      0.95      0.94       428
weighted avg       0.97      0.97      0.97       428

Feature Importance Ranking:
Width          0.337695
SnR            0.228965
Stdev          0.169109
KS_value       0.147378
Dip_value      0.075353
ESS            0.028785
Dip_p          0.008598
KS_p           0.004116
Upper_bound    0.000000
Lower_bound    0.000000
dtype: float64


/Users/justycewatson/Desktop/CIERA_files/BASE9_ML/create_model.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling_df['source_id'] = sampling_df['source_id'].astype(str)


In [8]:
X

,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.215020,4.260880e-253,10083.477879
1,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.157449,5.254432e-132,9752.934601
2,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.154297,6.979619e-130,9843.942167
3,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.139514,2.575757e-105,9649.833562
4,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.139253,2.198862e-102,9857.604416
...,...,...,...,...,...,...,...,...,...,...
1422,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.167510,1.076738e-150,9491.053710
1423,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.160163,9.946381e-138,10195.714180
1424,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.148133,7.255370e-116,9984.932919
1425,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.121893,4.047215e-79,9924.268626


In [24]:
model_cluster_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.216466,1.526297e-256,10362.245772
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.160905,7.015679e-138,9825.456950
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.151486,3.481028e-125,10224.951965
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.135951,5.256491e-100,9799.521154
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.143071,4.470739e-108,9629.116753
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.168245,5.043975e-152,9444.760628
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.160408,3.758991e-138,9667.750199
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.146805,8.522610e-114,9978.083278
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.128395,9.073887e-88,10105.480802


# Saving Model


- save model fn3

import pickle 
filename = 'my_model.pkl'  (any name .pkl)
pickle.dump(model, open(filename, 'wb'))   (saving)
loaded_model = pickle.load(open(filename, 'rb'))   (loading back in)

# Applying new dataset to saved model

- fn4 sending model to function, sending new dataset to it (6819) and returning the ml labels

Reusing first function to create stats df to make sure everything is exactly the same

- how to reuse first function and get the course ids correct? Or should I just load in file ? 

In [11]:
X_test

array([[ 0.3295017 ,  0.        ,  0.        , ...,  0.14025973,
        -0.04388632,  1.06411025],
       [ 0.36216461,  0.        ,  0.        , ..., -0.25698592,
        -0.04388632, -0.4799273 ],
       [-1.62262396,  0.        ,  0.        , ...,  0.14669994,
        -0.04388632,  1.69880429],
       ...,
       [ 0.58676434,  0.        ,  0.        , ...,  1.38644033,
        -0.04388632, -2.45992105],
       [-1.31238031,  0.        ,  0.        , ...,  0.31740493,
        -0.04388632, -1.6551812 ],
       [ 0.8261269 ,  0.        ,  0.        , ..., -0.62759382,
        -0.04388632,  0.22076502]])

In [5]:
directory = '/Users/justycewatson/Desktop/CIERA_files/NGC6819/ngc6819_single_resfiles' 

ngc6819_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
ngc6819_statistic

#need to send model same columns in xtest 
#convert columns from ngc6819 to np array -- send to make preds function

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,gaia_2076279137552154112_sin2,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.160000,8.402473e-141,9830.894809
1,gaia_2076298933067389056_sin2,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.256032,0.000000e+00,9913.750981
2,gaia_2076584355102288128_sin2,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.159919,2.001666e-138,9884.757613
3,gaia_2076488525792609792_sin2,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.285388,0.000000e+00,9804.369779
4,gaia_2076581851147039232_sin2,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.177733,4.139539e-171,10076.925082
...,...,...,...,...,...,...,...,...,...,...,...
1683,gaia_2076393834663325568_sin2,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.158725,1.462016e-138,9463.949929
1684,gaia_2076380400005524224_sin2,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.143401,2.877232e-111,9875.734491
1685,gaia_2076299895139969536_sin2,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.171406,2.222830e-160,10261.698367
1686,gaia_2076393662864610688_sin2,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.160398,1.661405e-141,9635.210562


In [6]:
ngc6819_clean, ngc6819_array = prepare_df_formodel(ngc6819_statistic)
ngc6819_array

array([[1.61110472e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.60000000e-001, 8.40247294e-141, 9.83089481e+003],
       [8.51659000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        2.56032389e-001, 0.00000000e+000, 9.91375098e+003],
       [1.56808700e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.59919028e-001, 2.00166589e-138, 9.88475761e+003],
       ...,
       [4.44381000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        1.71405622e-001, 2.22282996e-160, 1.02616984e+004],
       [1.51987300e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.60398406e-001, 1.66140476e-141, 9.63521056e+003],
       [5.23203000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        1.13333333e-001, 4.18312796e-070, 9.63839921e+003]])

In [40]:
ngc6819_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,gaia_2076279137552154112_sin2,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.162709,1.265698e-145,9648.646679
1,gaia_2076298933067389056_sin2,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.253117,0.000000e+00,9587.958798
2,gaia_2076584355102288128_sin2,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.157895,6.194576e-135,9852.001270
3,gaia_2076488525792609792_sin2,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.280163,0.000000e+00,9667.724432
4,gaia_2076581851147039232_sin2,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.177409,1.749204e-170,10018.253647
...,...,...,...,...,...,...,...,...,...,...,...
1683,gaia_2076393834663325568_sin2,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.163426,6.486647e-147,10062.901443
1684,gaia_2076380400005524224_sin2,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.135223,6.160077e-99,9944.673577
1685,gaia_2076299895139969536_sin2,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.166185,1.003523e-150,9893.809047
1686,gaia_2076393662864610688_sin2,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.161434,2.409169e-143,9880.907810


In [7]:
y_pred_6819 = make_preds(ngc6819_array, clf)
y_pred_6819

array(['Bad', 'Bad', 'Bad', ..., 'Bad', 'Bad', 'Bad'], dtype=object)